Sparl SQL and Dataframes essential docs:

- programming guide: https://spark.apache.org/docs/3.2.0/sql-programming-guide.html
- live notebook example: https://mybinder.org/v2/gh/apache/spark/5d45a415f3?filepath=python%2Fdocs%2Fsource%2Fgetting_started%2Fquickstart_df.ipynb
- example notebooks (git repo): https://github.com/apache/spark/tree/5d45a415f3/examples/src/main/python
- Dataframe API: https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html#dataframe-apis
- DataFrame Quickstart doc: https://spark.apache.org/docs/3.2.0/api/python/getting_started/quickstart_df.html

## Spark DataFrame and SQL API

- RDDs are (parallelised) collections of objects of some type.
- However, RDDs **do not have a schema**. This makes it difficult to perform typical "relational" operations.
- DataFrames have been introduced to facilitate relational operations, and to bring the abstraction level of Spark data closer to those who are more familiar with python / Pandas / relational tables programming.
- DataFrames are very similar to the corresponding concept eg in python / Pandas, however their content can be parallelised in the same way as that of RDD
- In fact, under the hood, a DataFrame's native native representation is as an RDD.

We now look at the relationship between DataFrames and RDDs

Suppose you are given a dataset containing taxi trips information.
Suppose each trip is represented as a record, for instance
`1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95`

this dataset can be loaded into a RDD, and Spark will parallelise it under the hood:

In [ ]:
tripsRDD = sc.textFile('/FileStore/tables/taxi/yellow_tripdata_2019_01_HEAD_1000_noheader.txt')

tripsRDD.take(5)

Out[1]: ['1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,',
 '1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,',
 '2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,.00,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,',
 '2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,',
 '2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,']

This RDD can be processed efficiently, however it does not carry any information about the meaning of the elements.

Suppose we know that the individual fields have the interpretation provided in the [NYC Taxi user guide](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page): 

`[VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge]`

**Exercise:** using `tripsRDD`, calculate the cost per mile `total_amount/trip_distance` for each trip, and then return the average of those values. Use `map()` and `reduce()` to achieve this

Suppose you are given a CSV file with the same data, complete with header:

`VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge`
`1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,`
`1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,`

here we have previously saved this as parquet file: `taxi/yellow_tripdata_2019_01_HEAD_1000.parquet`

We can load this file directly into a DataFrame, as follows:

In [ ]:
tripsDF = spark.read.parquet('/FileStore/tables/taxi/yellow_tripdata_2019_01_HEAD_1000.parquet')

this DataFrame carries column names and types, so it is effectively a relational table in the classic database sense. It is also consistent with Pandas DataFrames

if we try to replicate the exercise above, we find that the RDD operators `map()` and `reduce()` (and many others) are not available. 

However, the [DataFrame API](https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html#dataframe-apis) offers higher level operators that let us accomplish the same task easily.

The following 1-line code illustrates some of the key concepts in working with DataFrames

In [ ]:
## tripsDF['total_amount']/tripsDF['trip_distance']  operates on each row in the columns, and returns a spark.sql.Column object

## tripsDF.withColumn('unit_cost', <col>) adds a column defined by <col> named 'unit_cost' to the DataFrame

## <column>.groupby() constructs a group out of the entire <column>, returning a GroupedData  object which contains a single group with every value in the column

## <GroupedData>.mean(<col>) applies the built-in <mean> function to the values in <col> (for each group, in general)

## collect() triggers the entire computation and returns a value


tripsDF.withColumn('unit_cost', tripsDF['total_amount']/tripsDF['trip_distance']).groupBy().mean('unit_cost').collect()


Out[3]: [Row(avg(unit_cost)=7.636152863587667)]

you will notice that the result is expressed using a `Row` object. Each DataFrame has as underlying representation as RDD, where the type of the RDD is `spark.sql.Row`. 
This representation is accessed simply using

`df.rdd`:

In [ ]:
tripsDF.rdd

Out[5]: MapPartitionsRDD[16] at javaToPython at NativeMethodAccessorImpl.java:0

if you `collect()` or `take(n)` the content of the RDD, you can see the `Row` structure:

In [ ]:
tripsDF.rdd.take(2)

Out[7]: [Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2019, 1, 1, 0, 46, 40), tpep_dropoff_datetime=datetime.datetime(2019, 1, 1, 0, 53, 20), passenger_count=1, trip_distance=1.5, RatecodeID=1, store_and_fwd_flag='N', PULocationID=151, DOLocationID=239, payment_type=1, fare_amount=7.0, extra=0.5, mta_tax=0.5, tip_amount=1.65, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=9.95, congestion_surcharge=None),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2019, 1, 1, 0, 59, 47), tpep_dropoff_datetime=datetime.datetime(2019, 1, 1, 1, 18, 59), passenger_count=1, trip_distance=2.6, RatecodeID=1, store_and_fwd_flag='N', PULocationID=239, DOLocationID=246, payment_type=1, fare_amount=14.0, extra=0.5, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=16.3, congestion_surcharge=None)]

thus if you wanted to use `map()` for example, you could operate on `df.rdd`, with the caveat that you need to be aware that the data is now incapsulated into `Row` objects

individual fields in a Row can be accessed using `row.key` or `row[key]`

## Spark and Pandas dataframes

in the most recent version of Spark (3.2), you can also transform a Spark dataframe into a Pandas dataframe, and viceversa.

Spark provides an  API to interface with Pandas:
`import pyspark.pandas as ps` gives you access to Pandas DataFrames that are virtually interchangeable.

See [examples here](https://spark.apache.org/docs/3.2.0/api/python/user_guide/pandas_on_spark/pandas_pyspark.html) 
and [here](https://sparkbyexamples.com/pyspark/convert-pyspark-dataframe-to-pandas/)

Beware however, that if you simply use the nativa Pandas API:
`import pandas as pd` then you lose the benefit of distributed processing

## Spark SQL

since DataFrames are essentially relational tables, it makes sense to use SQL to operate on them.

This is accomplished using the spark.sql API, see here: https://spark.apache.org/docs/3.2.0/sql-getting-started.html#running-sql-queries-programmatically

the only requirement is that the DataFrame be mapped to a SQL table. This is done simply by:

`df.createOrReplaceTempView(<table name>)`

then you can use `<table name>` as a regular table name in a FROM clause in an SQL expression

In [ ]:
## compute the average unit cost for a trip:

tripsDF.createOrReplaceTempView('tripsTable')

sqlDF = spark.sql("SELECT avg(total_amount/trip_distance) as avg_unit_cost FROM tripsTable").show()

+-----------------+
 avg_unit_cost|
+-----------------+
7.636152863587667|
+-----------------+

**Exercise** 

calculate the average trip_distance for trips longer than 3 miles, for each of the possible payment types

In [ ]:
spark.sql("SELECT payment_type, avg(trip_distance) from tripsTable where trip_distance > 3 group by payment_type").show()

+------------+------------------+
payment_type|avg(trip_distance)|
+------------+------------------+
 1| 5.944899999999999|
 3| 15.1|
 2| 6.695229357798162|
+------------+------------------+